## Train Reinforcement Learning with  RLlIB

In [ ]:
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import PyTorchConfiguration


In [ ]:
#Remember the AML job has to have distribted setings (MPI type) for ray-on-aml to work correctly.
# See parameter distributed_job_config below
ws = Workspace.from_config()

compute_cluster = 'd15-v2' #This can be another cluster different from the interactive cluster. 
maxnode =2
vm_size='STANDARD_DS3_V2'
vnet='rayvnet'
subnet='default'
exp ='rl_on_aml_job'
ws_detail = ws.get_details()
ws_rg = ws_detail['id'].split("/")[4]
vnet_rg=None
try:
    ray_cluster = ComputeTarget(workspace=ws, name=compute_cluster)

    print('Found existing cluster, use it.')
except ComputeTargetException:
    if vnet_rg is None:
        vnet_rg = ws_rg
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                        min_nodes=0, max_nodes=maxnode,
                                                        vnet_resourcegroup_name=vnet_rg,
                                                        vnet_name=vnet,
                                                        subnet_name=subnet)
    ray_cluster = ComputeTarget.create(ws, compute_cluster, compute_config)

    ray_cluster.wait_for_completion(show_output=True)

#Check the conda_env.yml, it has an entry of ray-on-aml
rayEnv = Environment.from_conda_specification(name = "RLEnv",
                                             file_path = "ray_job_env.yml")



src = ScriptRunConfig(source_directory='job',
                script='rl_job.py',
                environment=rayEnv,
                compute_target=ray_cluster,
                distributed_job_config=PyTorchConfiguration(node_count=maxnode))
run = Experiment(ws, exp).submit(src)

In [ ]:
#Install additional library at Ray cluster
from azureml.widgets import RunDetails
RunDetails(run).show()